In [2]:
import json
import requests
import time
import re
import math
import collections
import random
try:
    from urllib.parse import parse_qs
except ImportError:
    from urlparse import parse_qs
HANGMAN_URL = "https://www.trexsim.com/trexsim/hangman"


In [3]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        full_dictionary_location = "./words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = self.full_dictionary
        self.idf = [0]*26
        self.incorrect_guess = []
        self.dict = {}
        self.original_dict = {}
        self.hangman_url = self.determine_hangman_url()

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']
        data = {link: 0 for link in links}

        for link in links:
            requests.get(link)
            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link    
    
    #finding the term frequency of all characters (a to z) in the list of words            
    def term_feq(self,words):
        ret = [0]*26
        for word in words:
            for letter in set(word):
                ret[ord(letter) - 97] +=self.dict[word] 
        for i in range(0,26):
            ret[i] = math.log(1 + ret[i],10)
        return ret   
   
    #This function generates n-grams of length n 
    def gen_n_gram(self,word, n):
        n_gram = []
        for i in range(n, len(word)+1):
            if word[i-n:i] not in n_gram:
                n_gram.append(word[i-n:i])
        return n_gram

    #returning the list of all the incorrect guesses till now.
    def find_incorrect_guess(self,word):
        ret = []
        for char in self.guessed_letters :
            if char not in word:
                ret.append(char)
        return ret

    def all_letters_in_word(self,letters,dict_word):
        for char in letters:
            if char not in dict_word:
                return False
        return True

    
    def matchingPhrase(self,phrase,dict_word):
        if len(phrase) != len(dict_word):
            return ""
        known_indexes = [i for i, ltr in enumerate(phrase) if ltr != "."]
        for i in known_indexes:
            if dict_word[i] != phrase[i]:
                return ""
        unknown_indexes = [i for i, ltr in enumerate(phrase) if ltr == "."]
        for i in unknown_indexes:
            if dict_word[i] in self.guessed_letters:
                return ""
        return dict_word

# return all the possible subproblems of length n of the current partial guessed word. 
    def all_possible_subproblems(self,word,n):
            ret = []
            if n > len(word):
                return ret
            n_gram = self.gen_n_gram(word, n)
            for allgrams in n_gram:
                unknown = find(allgrams,".")
                if len(unknown) > 0 :#and len(unknown) < len(allgrams)
                    ret.append(allgrams)
            return(ret)            

#check if a words any incorrectGuesses till now
    def containsIncorrect(self,word):
        
        for letter in self.incorrect_guess:
            if letter in word:
                return True
        return False        

#function to create our dictionary,all n-grams of length > 3 for all the words 
    def gen_all_n_grams(self):
        tokens = {}
        for word in self.full_dictionary:
            ngrams = []
            maxlen = len(word)-1
            for n in range(3,maxlen):
                ngrams = ngrams + self.gen_n_gram(word, n) 
                
            if word in tokens.keys():
                tokens[word] +=1
            else:
                tokens[word] = 1
            for token in ngrams:
                if token not in tokens.keys():
                    tokens[token]=1
                else:
                    tokens[token]+=1
        self.original_dict = tokens

# prune the dictionary by removing all the words with length greater than the word we need to guess
    def prune_dict(self,word):
        for dict_word in (self.original_dict.keys()):
            if len(word)<len(dict_word):
                del self.dict[dict_word]

            
    def guess(self, word): # word input example: "_ p p _ e "
                #cleaning the word            
                word = word[::2].replace("_",".")
                
                #pruning the dictionary when no guess has been made.
                if len(self.guessed_letters) == 0:
                    self.prune_dict(word)
                # populating the incorrect guesses till now.
                self.incorrect_guess = self.find_incorrect_guess(word)
                current_dictionary = list(self.dict.keys())
                new_dictionary = []
                n = len(word)
                while(len(new_dictionary) <= 10 and n >=3):
                    #finding all possible subproblems and its possible solutions 
                    # till we dont get at least 10 possible solutions. 
                    subproblems = self.all_possible_subproblems(word,n)
                    for dict_word in current_dictionary:
                        if len(dict_word)!=n :
                            continue
                        for problem in subproblems:
                            matchPhrase = self.matchingPhrase(problem,dict_word)
                            
                            if matchPhrase != "" and self.containsIncorrect(matchPhrase) == False:    
                                new_dictionary.append(dict_word)
                                #appending all possible solutions to our new_dictionary
                            else:
                                if matchPhrase!="":
                                    # deleting this word from the dictionary
                                    # as it already contains an ex
                                    del self.dict[dict_word]
                                
                    n-=1            
                #finding the term frequency of all 26 characters in the all possible solutions 
                tf = self.term_feq( new_dictionary)
                for i in range(0,26):
                    char = chr(i+97)
                    if char in self.guessed_letters:
                        tf[i]=(-1)
                
                # returning the max possible frequency letter 
                max_value = max(tf)
                max_index = tf.index(max_value)
                guess_letter = chr(max_index+97)
                return guess_letter

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
        
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        self.dict = self.original_dict.copy()                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
                
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        num_retry, time_sleep = 5, 2                                                                                        
        for it in range(num_retry):                                                                                         
            try:                                                                                                            
                response = self.session.request(                                                                            
                    method or "GET",                                                                                        
                    HANGMAN_URL + path,                                                                                     
                    timeout=self.timeout,                                                                                   
                    params=args,                                                                                            
                    data=post_args                                                                                          
                )                                                                                                           
                break                                                                                                       
            except requests.HTTPError as e:                                                                                 
                response = json.loads(e.read())                                                                             
                raise HangmanAPIError(response)                                                                             
            except requests.exceptions.SSLError as e:                                                                       
                if it + 1 == num_retry:                                                                                     
                    raise                                                                                                   
                time.sleep(time_sleep)  

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)


def intersection(str1,str2):
        res = ""
        for i in str1:
            if i in str2 and not i in res:
                res += i
        return res    
    
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]
  
def replace_str_index(text,index=0,replacement=''):
    return '%s%s%s'%(text[:index],replacement,text[index+1:])
    
                

            

In [10]:
#My simulator 
attempt = 0
correct_attempts = 0
correct_words = []
correct_guess = []

api = HangmanAPI(access_token="549f556bc32c3166631310e172e2e5", timeout=2000)
test = random.sample(api.full_dictionary, 1000 )
api.full_dictionary = set(api.full_dictionary) - set(test)
api.gen_all_n_grams()



while (attempt <100):
    #api = HangmanAPI(access_token="549f556bc32c3166631310e172e2e5", timeout=2000)
    api.current_dictionary = api.full_dictionary
    api.dict = api.original_dict.copy()
    api.guessed_letters = []
    api.incorrect_guess = []
    attempt+=1
    #word = random.choice(test)
    word = test[attempt]
        
    guessed_word = "_ "*len(word)
    guesses_left = 6
    while(guesses_left>0):
        guess = api.guess(guessed_word)        
        guesses_left-=1
        indexes = []
        api.guessed_letters.append(guess)
        if guess in word:
            guesses_left+=1
            indexes = find(word,guess)
        else:
            api.incorrect_guess.append(guess)
        for i in indexes:
            guessed_word = replace_str_index(guessed_word,(i+1)*2 -2 ,guess)
        print(attempt, correct_attempts,word, guessed_word, guess,guesses_left)
        if guessed_word.replace(" ", "") == word:
            break
        
        
        
    #checking for correctness
    if ((word == guessed_word.replace(" ", "")) and guesses_left >=0 ):
        correct_attempts+=1
        correct_words.append(word)
        correct_guess.append(guessed_word.replace(" ", ""))
    
#str1 = "abcasdsadwq"
#print("".join(set(str1)))

print(correct_attempts)
#print(correct_words)
#print(correct_guess)

1 0 ethicoreligious _ _ _ i _ _ _ _ _ i _ i _ _ _  i 6
1 0 ethicoreligious _ _ _ i c _ _ _ _ i _ i _ _ _  c 6
1 0 ethicoreligious _ _ _ i c _ _ _ l i _ i _ _ _  l 6
1 0 ethicoreligious e _ _ i c _ _ e l i _ i _ _ _  e 6
1 0 ethicoreligious e _ _ i c _ _ e l i g i _ _ _  g 6
1 0 ethicoreligious e _ _ i c o _ e l i g i o _ _  o 6
1 0 ethicoreligious e _ _ i c o r e l i g i o _ _  r 6
1 0 ethicoreligious e _ _ i c o r e l i g i o u _  u 6
1 0 ethicoreligious e _ _ i c o r e l i g i o u s  s 6
1 0 ethicoreligious e _ h i c o r e l i g i o u s  h 6
1 0 ethicoreligious e _ h i c o r e l i g i o u s  p 5
1 0 ethicoreligious e t h i c o r e l i g i o u s  t 5
2 1 frache _ _ _ _ _ e  e 6
2 1 frache _ _ _ _ _ e  i 5
2 1 frache _ _ a _ _ e  a 5
2 1 frache _ _ a _ _ e  l 4
2 1 frache _ r a _ _ e  r 4
2 1 frache _ r a _ _ e  g 3
2 1 frache _ r a c _ e  c 3
2 1 frache _ r a c _ e  t 2
2 1 frache _ r a c h e  h 2
2 1 frache _ r a c h e  b 1
2 1 frache _ r a c h e  o 0
3 1 campaniform _ _ _ _ _ _ i _ 

In [5]:
api = HangmanAPI(access_token="549f556bc32c3166631310e172e2e5", timeout=2000)

In [4]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game


NameError: name 'api' is not defined

In [7]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.004
